Copyright 2024 - Forusone : shins777@gmail.com

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

   https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

# Build Cloud Function serving advanced RAG archiectures.


## Set working directory and mount Google drive

In [ ]:
#  For only colab
import sys

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

    # To access contents in Google drive
    from google.colab import drive
    drive.mount('/content/drive')

In [ ]:
#  source root directory
src_dir = '/content/drive/MyDrive/projects/genai_workshop/notebook/03.cloud function/functions'

# cloud function src directory.
import os
os.chdir(src_dir)
!pwd

## Source code for Cloud function

In [ ]:
from google.colab import files
files.view(f'{src_dir}/main.py')
files.view(f'{src_dir}/requirements.txt')
files.view(f'{src_dir}/rag/constant.py')
files.view(f'{src_dir}/rag/controller.py')

In [ ]:
#%%writefile table_search.py

## Authentication

In [ ]:
!gcloud auth application-default login
!gcloud config set project ai-hangsik
!gcloud auth application-default set-quota-project ai-hangsik

In [ ]:
!pwd

## Deploy Cloud Run - Text Processing

In [ ]:
!gcloud functions deploy it_search --allow-unauthenticated \
	--region=asia-northeast3 \
	--gen2 \
	--memory=8Gi \
	--cpu=4 \
	--concurrency=4 \
	--min-instances=2 \
	--max-instances=5 \
	--timeout=30s \
	--runtime=python312 \
  --trigger-http \
	--source=. \
	--entry-point=search


## REST API Call to Cloud function.

In [ ]:
!curl -X POST https://it-search-721521243942.asia-northeast3.run.app \
-H 'Content-Type: application/json' \
-d '{"question":"개인정보 보호법에 대해서 설명해주세요."}'

